## Load BOM

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import time as t
import json
from tqdm import tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# from dotenv import load_dotenv
# load_dotenv()

# List Type: Values, Output Format: CSV
# Make sure no semi-colons are present in part descriptions
BOM_path = f"../AK4493-v1.64_26.csv"

BOM_df = pd.read_csv(BOM_path, delimiter = ";")
search_string = 'Unnamed'
columns_containing_string = [col for col in BOM_df.columns if search_string in col]
BOM_df.drop(columns = columns_containing_string, inplace = True)
display(BOM_df)
print("Done")

,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,BOM,DESCRIPTION,HEIGHT,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,POPULARITY,PROD_ID,SPICEPREFIX
0,3,NaN,CONN_02,1X02,"J1, J2, J14",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J4, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437,NaN
2,9,NaN,CONN_07,1X07,"J3, J5, J6, J7, J8, J9, J10, J11, J12",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Combine 8288+14485,NaN
3,2,NaN,"CONN_10"";1X10""","J21, J22",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-11563,NaN,NaN
4,2,NaN,SJ,SJ,"SMUTE1, SMUTE2",SMD solder JUMPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
5,2,NaN,SJ2W,SJ_2,"DIF1, DIF3",SMD solder JUMPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
6,6,0.1u,EMF212B7104KGHT,CAPC2012X135N,"C21A, C21B, C21C, C21D, C21E, C21F",Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,NaN,NaN,NaN,Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,1.35mm,TAIYO YUDEN,EMF212B7104KGHT,963-EMF212B7104KGHT,https://www.mouser.co.uk/ProductDetail/Taiyo-Y...,NaN,NaN,NaN
7,5,"0.1u, Ceramic Capacitor X7R",EMF212B7104KGHT,CAPC2012X135N,"C22A, C23A, C24A, C28A, C28B",Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,NaN,NaN,NaN,Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,1.35mm,TAIYO YUDEN,EMF212B7104KGHT,963-EMF212B7104KGHT,https://www.mouser.co.uk/ProductDetail/Taiyo-Y...,NaN,NaN,NaN
8,16,"0.1u, Ceramic Capacitor X7R",EMK107B7104KA-T,CAPC1608X95N,"C2A1, C2A2, C4A1, C4A2, C6A1, C6A2, C8A1, C8A2...",EMK107B7104KA-T 0.1F +/-10% 16V Ceramic Capaci...,NaN,NaN,NaN,EMK107B7104KA-T 0.1F +/-10% 16V Ceramic Capaci...,0.95mm,TAIYO YUDEN,EMK107B7104KA-T,963-EMK107B7104KA-T,https://www.mouser.co.uk/ProductDetail/TAIYO-Y...,NaN,NaN,NaN
9,1,"0.1uF, X7R",EMF212B7104KGHT,CAPC2012X135N,C1A,Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,NaN,NaN,NaN,Multilayer Ceramic Capacitors MLCC - SMD/SMT 0...,1.35mm,TAIYO YUDEN,EMF212B7104KGHT,963-EMF212B7104KGHT,https://www.mouser.co.uk/ProductDetail/Taiyo-Y...,NaN,NaN,NaN


Done


## Define search_part function

In [6]:
def search_part(part_name):
    MOUSER_API_KEY = "c7b47461-c777-4dbc-8413-9bb69528b78f"
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    params = {
        'apiKey': MOUSER_API_KEY,
    }
    
    json_data = {
        'SearchByPartRequest': {
            'mouserPartNumber': part_name,
            'partSearchOptions': 'string',
            'mouserPaysCustomsAndDuties': False,
        },
    }

    try_again_flag = True
    max_attempts = 10
    attempts = 0
    
    while try_again_flag and attempts < max_attempts:
        response = requests.post('https://api.mouser.com/api/v1/search/partnumber', params=params, headers=headers, json=json_data)
    
        t.sleep(6)
    
        match response.status_code:
            case 200:
                data = response.json()
                try_again_flag = False
            case 404:
                print("Resource not found (404 Not Found).")
                attempts += 1
            case 403:
                print("Permission denied (403 Forbidden).")
                attempts += 1
            case response.status_code if 300 <= response.status_code < 400:
                print(f"Redirection occurred (Status: {response.status_code}).")
                attempts += 1
            case response.status_code if 500 <= response.status_code < 600:
                print(f"Server error occurred (Status: {response.status_code}).")
                attempts += 1
            case _:
                print(f"Unexpected status code: {response.status_code} - {response.text}")
                attempts += 1

    if attempts == max_attempts:
        print(f"Max attempts reached - {attempts} / {max_attempts}.")
        return
    
    df = pd.DataFrame()
    
    # Sort and append new data
    for part in data["SearchResults"]["Parts"]:
        df = pd.concat([df, pd.json_normalize(part)], ignore_index=True)

    columns_to_drop = ["ImagePath",
                     "AlternatePackagings",
                     "ProductDetailUrl",
                     "Reeling",
                     "MultiSimBlue",
                     "InfoMessages",
                     "SurchargeMessages",
                     "ProductCompliance",
                     "REACH-SVHC",
                     "RestrictionMessage",
                     "ROHSStatus",
                     "ProductAttributes",
                     "FactoryStock",
                     "AvailabilityOnOrder",
                     "UnitWeightKg.UnitWeight",
                    ]

    for column_to_drop in columns_to_drop:
        if column_to_drop in df.columns:
            df.drop([column_to_drop], axis = 1, inplace = True)
        else:
            pass 

    if ("PriceBreaks" in df.columns) and not (df["PriceBreaks"].apply(lambda x: len(x) == 0).all()):
        df_exploded = df.explode("PriceBreaks").reset_index(drop=True)
        pricing_df = pd.json_normalize(df_exploded["PriceBreaks"])
        df_combined = pd.concat([df_exploded.drop(columns=["PriceBreaks"]), pricing_df], axis=1)
        df = df_combined.set_index(["MouserPartNumber", "Quantity"]).sort_index()
    else:
        return pd.DataFrame()
    
    for column in df.columns:
        try:
            df[column] = pd.to_numeric(df[column])
        except Exception as e:
            pass
    
    df = df[df["AvailabilityInStock"] > 0]
    df = df[df["Min"] == 1]
    df = df[df["Mult"] == 1]    
    df = df.sort_values(by=['Quantity', 'Price'])
    
    if len(df.index.get_level_values(0)) > 0:
        return df.loc[[df.index.get_level_values(0)[0]]]
    else:
        return pd.DataFrame()

print("Done")

Done


## Search Parts

In [7]:
results = pd.DataFrame()
pared_BOM = pd.DataFrame()
missing_parts = pd.DataFrame()

with tqdm(total = len(BOM_df["MOUSER_PART_NUMBER"].values), unit = "part") as pbar:
    for idx, row in BOM_df.iterrows():
        if row["MOUSER_PART_NUMBER"] is not np.nan:
            part_result = search_part(row["MOUSER_PART_NUMBER"])
        else:
            part_result = search_part(row["Device"])
            
        if part_result.empty:
            missing_parts = pd.concat([missing_parts, row], axis = 1)
        else:
            results = pd.concat([results, pd.concat([part_result.reset_index(), pd.concat([pd.DataFrame(row).T] * len(part_result), ignore_index=True)], axis = 1)], axis = 0)
        pbar.update(1)

results = results.set_index(["MouserPartNumber", "Quantity"]).sort_index()
display(results)
print("Cant find part(s):")
display(missing_parts.T)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [03:51<00:00,  6.62s/part]


Availability DataSheetUrl  \
MouserPartNumber     Quantity                                 
279-CPF0805B51KE1    1           2236 In Stock          NaN   
                     10          2236 In Stock          NaN   
                     100         2236 In Stock          NaN   
                     500         2236 In Stock          NaN   
                     1000        2236 In Stock          NaN   
...                                        ...          ...   
963-EMK316BB7226ML-T 1000      114797 In Stock          NaN   
                     2000      114797 In Stock          NaN   
                     4000      114797 In Stock          NaN   
                     10000     114797 In Stock          NaN   
                     24000     114797 In Stock          NaN   

                                                                     Description  \
MouserPartNumber     Quantity                                                      
279-CPF0805B51KE1    1         Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     10        Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     100       Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     500       Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
                     1000      Thin Film Resistors - SMD CPF 0805 51K 0.1% 25PPM   
...                                                                          ...   
963-EMK316BB7226ML-T 1000      Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     2000      Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     4000      Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     10000     Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   
                     24000     Multilayer Ceramic Capacitors MLCC - SMD/SMT 9...   

                                                                   Category  \
MouserPartNumber     Quantity                                                 
279-CPF0805B51KE1    1                            Thin Film Resistors - SMD   
                     10                           Thin Film Resistors - SMD   
                     100                          Thin Film Resistors - SMD   
                     500                          Thin Film Resistors - SMD   
                     1000                         Thin Film Resistors - SMD   
...                                                                     ...   
963-EMK316BB7226ML-T 1000      Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     2000      Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     4000      Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     10000     Multilayer Ceramic Capacitors MLCC - SMD/SMT   
                     24000     Multilayer Ceramic Capacitors MLCC - SMD/SMT   

                               LeadTime LifecycleStatus  \
MouserPartNumber     Quantity                             
279-CPF0805B51KE1    1         101 Days             NaN   
                     10        101 Days             NaN   
                     100       101 Days             NaN   
                     500       101 Days             NaN   
                     1000      101 Days             NaN   
...                                 ...             ...   
963-EMK316BB7226ML-T 1000       91 Days             NaN   
                     2000       91 Days             NaN   
                     4000       91 Days             NaN   
                     10000      91 Days             NaN   
                     24000      91 Days             NaN   

                                               Manufacturer  \
MouserPartNumber     Quantity                                 
279-CPF0805B51KE1    1         TE Connectivity / Holsworthy   
                     10        TE Connectivity / Holsworthy   
                     100       TE Connectivity / Holsworthy   
          

Cant find part(s):


,Qty,Value,Device,Package,Parts,Description,ARROW_PART_NUMBER,ARROW_PRICE-STOCK,BOM,DESCRIPTION,HEIGHT,MANUFACTURER_NAME,MANUFACTURER_PART_NUMBER,MOUSER_PART_NUMBER,MOUSER_PRICE-STOCK,POPULARITY,PROD_ID,SPICEPREFIX
0,3,NaN,CONN_02,1X02,"J1, J2, J14",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,CONN_06SILK_FEMALE_PTH,1X06,"J4, J13",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-08437,NaN
2,9,NaN,CONN_07,1X07,"J3, J5, J6, J7, J8, J9, J10, J11, J12",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Combine 8288+14485,NaN
3,2,NaN,"CONN_10"";1X10""","J21, J22",Multi connection point. Often used as Generic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONN-11563,NaN,NaN
4,2,NaN,SJ,SJ,"SMUTE1, SMUTE2",SMD solder JUMPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
5,2,NaN,SJ2W,SJ_2,"DIF1, DIF3",SMD solder JUMPER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
15,1,"2200uF, 10V",APSG160ELL222MJ20S,CAPPRD500W60D1025H2150,C11B,Aluminum Organic Polymer Capacitors 16V 2200uF...,NaN,NaN,NaN,Aluminum Organic Polymer Capacitors 16V 2200uF...,21.5mm,Chemi-Con,APSG160ELL222MJ20S,661-APSG160E222MJ20S,https://www.mouser.co.uk/ProductDetail/United-...,NaN,NaN,NaN
25,2,APSG160ELL222MJ20S,APSG160ELL222MJ20S,CAPPRD500W60D1025H2150,"C7B, C11A",Aluminum Organic Polymer Capacitors 16V 2200uF...,NaN,NaN,NaN,Aluminum Organic Polymer Capacitors 16V 2200uF...,21.5mm,Chemi-Con,APSG160ELL222MJ20S,661-APSG160E222MJ20S,https://www.mouser.co.uk/ProductDetail/United-...,NaN,NaN,NaN
26,1,I2SOVERUSBV3,I2SOVERUSBV3,I2SOVERUSBV3,BD1,Asynchronous audio transfer at sampling rates ...,NaN,NaN,Yes,Asynchronous audio transfer at sampling rates ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,1,Not Pop.,R-POLR0805,R0805-POL,R21A,"RESISTOR, American symbol",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,R


## Create Output Dataframe

In [18]:
quantities = np.unique([j for i, j in results.index.values])
quantities = quantities[quantities <= 100]

price_breaks = pd.DataFrame(np.unique([i for i, j in results.index.values]), columns = ["MouserPartNumber"])
price_breaks.set_index("MouserPartNumber", inplace = True)

for part_num in [i for i, j in results.index.values]:
    for quantity in quantities:
        result_df = results.loc[(results.index.get_level_values('MouserPartNumber') == part_num) & (results.index.get_level_values('Quantity') == quantity)]["Price"]
        if result_df.empty:
            price_breaks.at[part_num, f"Unit Price Per {quantity}"] = np.nan
        else:
            if len(result_df.values) == 1:
                price_breaks.at[part_num, f"Unit Price Per {quantity}"] = result_df.values
            elif len(set(result_df.values)) == 1:
                price_breaks.at[part_num, f"Unit Price Per {quantity}"] = result_df.values[0]
            
price_breaks = price_breaks.ffill(axis=1)

# Remove dollar signs
for idx, row in price_breaks.iterrows(): # Bear in mind that "idx" is the part name!!!
    for col_name in price_breaks.columns:
        row[col_name] = str(row[col_name]).replace('$', '')

for column in price_breaks.columns:
    try:
        price_breaks[column] = pd.to_numeric(price_breaks[column])
    except Exception as e:
        pass

# Add in columns from BOM
in_cols = ["Qty", "Value", "Device"]
out_cols = ["Per 1 Qty", "Value", "Device"]

for idx1, (in_col, out_col) in enumerate(zip(in_cols, out_cols)):
    for idx2, part_num in enumerate(price_breaks.index.values):
        qty_val = np.unique(results.loc[results.index.get_level_values('MouserPartNumber') == part_num][in_col].values)[0]
    
        if idx2 == 0:
            price_breaks.insert(loc = idx1, column = out_col, value = qty_val)
        else:
            price_breaks.at[part_num, out_col] = qty_val

# Qty
multiplier = 1
price_breaks.insert(loc = 1, column = "Qty", value = [i * multiplier for i in price_breaks["Per 1 Qty"].values])

# Extras?
extras = np.full((len(price_breaks)), True)
extras[-5] = False
price_breaks.at[:, "Extras?"] = extras

# Sub-count
for idx, row in price_breaks.iterrows():
    if row["Extras?"] and (float(row["Unit Price Per 1"]) < 1):
        price_breaks.at[idx, "Sub-count"] = np.round(row["Qty"] + 1 + row["Qty"] * 0.03)
    else:
        price_breaks.at[idx, "Sub-count"] = row["Qty"]

# Sub-totals per number
[f"Sub-total Per {quantity}" for quantity in quantities]
in_cols = [f"Unit Price Per {quantity}" for quantity in quantities]
out_cols = [f"Sub-total Per {quantity}" for quantity in quantities]

for _, (in_col, out_col) in enumerate(zip(in_cols, out_cols)):
    for idx, row in price_breaks.iterrows():
        multiplier = int(in_col.replace("Unit Price Per ", ""))
        if row["Sub-count"] > (multiplier - 1):
            price_breaks.at[idx, out_col] = row["Sub-count"] * row[in_col]
        else:
            price_breaks.at[idx, out_col] = multiplier *  row[in_col]

# Final Count
for idx, row in price_breaks.iterrows():
    # multiplier = int(in_col.replace("Unit Price Per ", ""))
    if row["Sub-count"] < int(row[out_cols].idxmin().replace("Sub-total Per ", "")):
        price_breaks.at[idx, "Final Count"] = int(row[out_cols].idxmin().replace("Sub-total Per ", ""))
    else:
        price_breaks.at[idx, "Final Count"] = row["Sub-count"]

# Final Sub-total
for idx, row in price_breaks.iterrows():
    price_breaks.at[idx, "Final Sub-total"] = price_breaks.at[idx, "Final Count"] * price_breaks.at[idx, f"Unit Price Per {int(row[out_cols].idxmin().replace("Sub-total Per ", ""))}"]

display(price_breaks)
print(f"Merchandise Total: ${np.sum(price_breaks["Final Sub-total"])}")

,Per 1 Qty,Qty,Value,Device,Unit Price Per 1,Unit Price Per 10,Unit Price Per 25,Unit Price Per 50,Unit Price Per 60,Unit Price Per 100,Extras?,Sub-count,Sub-total Per 1,Sub-total Per 10,Sub-total Per 25,Sub-total Per 50,Sub-total Per 60,Sub-total Per 100,Final Count,Final Sub-total
MouserPartNumber,,,,,,,,,,,,,,,,,,,,
279-CPF0805B51KE1,6,6,33k,CPF0805B51KE1,0.25,0.171,0.171,0.171,0.171,0.153,True,7.0,1.75,1.710,4.275,8.55,10.26,15.3,10.0,1.710
412-AK4493SEQ,2,2,AK4493,AK4493,8.93,6.940,6.450,6.450,6.450,5.900,True,2.0,17.86,69.400,161.250,322.50,387.00,590.0,2.0,17.860
584-LT3042EMSE#PBF,6,6,LT3042,LT3042,7.29,5.090,5.090,4.200,4.200,3.980,True,6.0,43.74,50.900,127.250,210.00,252.00,398.0,6.0,43.740
595-LP5907MFX-1.8NPB,2,2,LP5907-1.8V,LP5907-1.8V,0.54,0.386,0.347,0.347,0.347,0.307,True,3.0,1.62,3.860,8.675,17.35,20.82,30.7,3.0,1.620
595-SN74ABT273DBR,1,1,SN74ABT273DBR,SN74ABT273DBR,0.92,0.662,0.596,0.596,0.596,0.525,True,2.0,1.84,6.620,14.900,29.80,35.76,52.5,2.0,1.840
603-RT0603DRE075R1L,8,8,RT0603DRE075R1L,RT0603DRE075R1L,0.10,0.024,0.024,0.024,0.024,0.018,True,9.0,0.90,0.240,0.600,1.20,1.44,1.8,10.0,0.240
647-RNE1C471MDN,3,3,470u,RNE1C471MDN1,1.20,0.718,0.718,0.718,0.718,0.650,True,3.0,3.60,7.180,17.950,35.90,43.08,65.0,3.0,3.600
647-RNU1A100MDSASQ,1,1,647-RNU1A100MDSASQ,RNU1A100MDSASQ,0.86,0.574,0.574,0.574,0.574,0.414,True,2.0,1.72,5.740,14.350,28.70,34.44,41.4,2.0,1.720
652-CHP0805FX1001ELF,6,6,1k,CHP0805-FX-1001ELF,0.10,0.075,0.075,0.075,0.075,0.072,True,7.0,0.70,0.750,1.875,3.75,4.50,7.2,7.0,0.700


Merchandise Total: $87.57600000000001


## Output in Mouser Part List Import Tool Format

In [19]:
for idx, row in price_breaks.iterrows():
    print(row.name + "|" + str(int(row["Final Count"])))

279-CPF0805B51KE1|10
412-AK4493SEQ|2
584-LT3042EMSE#PBF|6
595-LP5907MFX-1.8NPB|3
595-SN74ABT273DBR|2
603-RT0603DRE075R1L|10
647-RNE1C471MDN|3
647-RNU1A100MDSASQ|2
652-CHP0805FX1001ELF|7
653-XR2A-0811-N|7
71-RCS080510K0FKEA|2
71-RCS08052K20FKEA|3
754-RR1220P-204D|10
80-A758BG106M1EAAE70|4
963-EMF212B7104KGHT|1
963-EMK107B7104KA-T|17
963-EMK212ABJ475KG-T|10
963-EMK212BB7106KG-T|10
963-EMK316BB7226ML-T|10
